<a href="https://colab.research.google.com/github/rishantmallick/Backtester/blob/main/univariate_backtesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openpyxl
!pip install backtrader
!pip install pandas_ta
!pip install pandas
!pip install yfinance
!pip install ddgs
!pip install mplfinance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.5/419.5 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.3/240.3 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 99.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 116.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 113.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 17.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.43.0
    Uninstalling llvmlite-0.43.0:
      Successfully uninstalled llvmlite-0.43.0
  Attempting uninstall: pandas
    Found existing i

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 90.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 3.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mplfinance as mpf
def compute_macd(data, fast=12, slow=26, signal=9):
    prices = data['Close']
    ema_fast = prices.ewm(span=fast, adjust=False).mean()
    ema_slow = prices.ewm(span=slow, adjust=False).mean()
    macd_line = ema_fast - ema_slow
    signal_line = macd_line.ewm(span=signal, adjust=False).mean()
    histogram = macd_line - signal_line
    return (macd_line,signal_line,histogram)
def compute_atr(data, period=2):
    high = data['High']
    low = data['Low']
    close = data['Close']
    high_low = high - low
    high_close = (high - close.shift()).abs()
    low_close = (low - close.shift()).abs()
    tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    atr = tr.ewm(span=14, adjust=False).mean()
    return atr

def compute_rsi(data, period=2):
    prices = data['Close']
    delta = prices.diff()
    gain = delta.where(delta > 0, 0.0)
    loss = -delta.where(delta < 0, 0.0)

    avg_gain = gain.rolling(window=period).mean()
    avg_loss = loss.rolling(window=period).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    return rsi
def compute_bollinger_bands(data):
  length = 3
  std= 2
  prices = data['Close']
  middle = prices.rolling(window=length).mean()
  rolling_std = prices.rolling(window=length).std()
  upper = middle + std * rolling_std
  lower = middle - std * rolling_std
  return (upper,lower,middle)
def obv_manual(df):
    obv = [0]  # start OBV at 0
    for i in range(1, len(df)):
        if df['Close'].iloc[i] > df['Close'].iloc[i-1]:
            obv.append(obv[-1] + df['Volume'].iloc[i])
        elif df['Close'].iloc[i] < df['Close'].iloc[i-1]:
            obv.append(obv[-1] - df['Volume'].iloc[i])
        else:
            obv.append(obv[-1])
    return obv

In [ ]:
def generate_trading_signals(data):
  macd_signal,signal_line,histogram = compute_macd(data)
  rsi = compute_rsi(data)
  upper,lower,middle = compute_bollinger_bands(data)
  atr = compute_atr(data)
  atr_min = 0.8
  atr_max = 2.0
  atr_sma = atr.rolling(window=14).mean()
  atr_min_threshold = atr_sma * atr_min
  atr_max_threshold = atr_sma * atr_max
  signal=[]
  ema_5 = data['Close'].ewm(span=5, adjust=False).mean()
  ema_21 = data['Close'].ewm(span=21, adjust=False).mean()
  for i in range(len(data['Close'].values.flatten())):
    if atr.values.flatten()[i] >= atr_min_threshold.values.flatten()[i] and atr.values.flatten()[i] <= atr_max_threshold.values.flatten()[i]:
      score = 0
      #BUY
      if rsi.values.flatten()[i] < 30 :
         score += 3
      elif rsi.values.flatten()[i] < 40 :
         score += 2
      elif rsi.values.flatten()[i] < 50 :
         score += 1
      else:
         score += 0

      if macd_signal.values.flatten()[i] > signal_line.values.flatten()[i] :
        if i>0 and macd_signal.values.flatten()[i-1] < signal_line.values.flatten()[i-1] :
          score += 3
        else:
          score += 1

      if data['Close'].values.flatten()[i] < lower.values.flatten()[i] :
        score += 2
      elif data['Close'].values.flatten()[i] < middle.values.flatten()[i] :
        score += 1
      else:
        score += 0

      if ema_5.values.flatten()[i] > ema_21.values.flatten()[i] :
        score += 1

      if i >= 2:
            if histogram.values.flatten()[i] > histogram.values.flatten()[i-1] > histogram.values.flatten()[i-2]:
                score += 1  # Accelerating bullish momentum

      #SELL
      if rsi.values.flatten()[i] > 70 :
         score += -3
      elif rsi.values.flatten()[i] > 65 :
         score += -2
      elif rsi.values.flatten()[i] > 55 :
         score += -1
      else:
         score += 0

      if macd_signal.values.flatten()[i] < signal_line.values.flatten()[i] :
        if i>0 and macd_signal.values.flatten()[i-1] > signal_line.values.flatten()[i-1] :
          score += -3
        else:
          score += -1

      if data['Close'].values.flatten()[i] > upper.values.flatten()[i] :
        score += -2
      elif data['Close'].values.flatten()[i] > middle.values.flatten()[i] :
        score += -1
      else:
        score += 0

      if ema_5.values.flatten()[i] < ema_21.values.flatten()[i] :
        score += -1

      if i >= 2:
            if histogram.values.flatten()[i] < histogram.values.flatten()[i-1] < histogram.values.flatten()[i-2]:
                score -= 1  # Accelerating bearish momentum

      if score > 0 :
        signal.append('Buy')
      elif score < 0 :
        signal.append('Sell')
      else :
        signal.append('Hold')
    else:
      signal.append('Hold')
  return signal

In [ ]:
import yfinance as yf
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
df = yf.download("AAPL",start="2015-01-01",end="2026-01-01",progress=False)
df = df.reset_index()
df = df[["Date", "Close", "High", "Low", "Open", "Volume"]]
data = df
dg = generate_trading_signals(data)
print(dg)
data['status']  = dg
datas = {
    "Date" : data['Date'].values,
    "Open" : data['Open'].values.flatten(),
    "High" : data['High'].values.flatten(),
    "Low" : data['Low'].values.flatten(),
    "Close" : data['Close'].values.flatten(),
    "Volume" : data['Volume'].values.flatten(),
    "status" : data['status'].values
}
df = pd.DataFrame(datas)
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace = True)
status_map = {'Hold': 0, 'Buy': 1, 'Sell': -1}
df['status'] = df['status'].map(status_map)
print(df)

/usr/local/lib/python3.12/dist-packages/yfinance/scrapers/history.py:204: Pandas4Warning: Timestamp.utcnow is deprecated and will be removed in a future version. Use Timestamp.now('UTC') instead.
  dt_now = pd.Timestamp.utcnow()


['Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Buy', 'Sell', 'Sell', 'Buy', 'Hold', 'Sell', 'Hold', 'Buy', 'Sell', 'Sell', 'Sell', 'Buy', 'Buy', 'Sell', 'Sell', 'Sell', 'Sell', 'Hold', 'Hold', 'Hold', 'Hold', 'Sell', 'Sell', 'Buy', 'Buy', 'Sell', 'Buy', 'Sell', 'Buy', 'Buy', 'Buy', 'Sell', 'Buy', 'Buy', 'Sell', 'Sell', 'Sell', 'Sell', 'Sell', 'Buy', 'Buy', 'Hold', 'Sell', 'Buy', 'Buy', 'Sell', 'Sell', 'Sell', 'Buy', 'Sell', 'Sell', 'Buy', 'Buy', 'Sell', 'Buy', 'Hold', 'Buy', 'Buy', 'Hold', 'Sell', 'Sell', 'Buy', 'Sell', 'Sell', 'Sell', 'Sell', 'Buy', 'Buy', 'Buy', 'Sell', 'Sell', 'Buy', 'Buy', 'Sell', 'Sell', 'Sell', 'Buy', 'Buy', 'Sell', 'Buy', 'Sell', 'Sell', 'Buy', 'Sell', 'Sell', 'Buy', 'Buy', 'Sell', 'Buy', 'Buy', 'Buy', 'Buy', 'Buy', 'Buy', 'Buy', 'Buy', 'Sell', 'Sell', 'Buy', 'Buy', 'Sell', 'Sell', 'Sell', 'Buy', 'Sell', 'Sell', 'Sell', 'Sell', 'Buy', 'Buy', 'Buy', 'Sell', 'Sell', 'Buy', 'Buy', 'Buy', 'Buy', 'Sell', 'Sel

In [ ]:
import backtrader as bt
def backtest_univariate_trading(data):
  class PandasDataCustom(bt.feeds.PandasData):
    lines = ('status',)
    params = (
    ('status', 'status'),
    ('open', 'Open'),
    ('high', 'High'),
    ('low', 'Low'),
    ('close', 'Close'),
    ('volume', 'Volume'),
    ('datetime', None),
    ('openinterest', -1)
    )


  class MyStrategy(bt.Strategy):
    params = (
        ('min_signal_duration', 3),  # Require signal to persist for N bars
        ('use_filter', True),        # Filter out weak signals
        ('filter_threshold', 0.3),   # Minimum signal strength
        ('max_trades_per_day', 2),   # Limit overtrading
        ('use_trailing_stop', True), # Protect profits
        ('trail_percent', 0.005),    # 0.5% trailing stop
    )

    def __init__(self):
        self.signal_streak = 0
        self.last_signal = 0
        self.trades_today = 0
        self.last_trade_date = None

        # Optional: Add indicators to filter signals
        self.sma = bt.indicators.SMA(self.data.close, period=20)
        self.atr = bt.indicators.ATR(self.data, period=14)

    def next(self):
        current_date = self.data.datetime.date(0)

        # Reset daily trade counter
        if current_date != self.last_trade_date:
            self.trades_today = 0
            self.last_trade_date = current_date

        signal = self.datas[0].status[0]

        # Track signal persistence
        if signal == self.last_signal:
            self.signal_streak += 1
        else:
            self.signal_streak = 1
            self.last_signal = signal

        # Don't trade if already at daily limit
        if self.trades_today >= self.p.max_trades_per_day:
            return

        # ADD FILTERS HERE
        is_trend_aligned = True
        if self.p.use_filter:
            # Example filter: Only trade in direction of trend
            if signal == 1:
                is_trend_aligned = self.data.close[0] > self.sma[0]
            elif signal == -1:
                is_trend_aligned = self.data.close[0] < self.sma[0]

        # REQUIRE SIGNAL PERSISTENCE
        signal_valid = (self.signal_streak >= self.p.min_signal_duration)

        # ENTRY LOGIC WITH FILTERS
        if signal == 1 and signal_valid and is_trend_aligned:
            if not self.position:
                # Calculate position size based on ATR
                atr_value = self.atr[0]
                price = self.data.close[0]
                risk_per_share = atr_value * 1.5
                risk_amount = self.broker.getvalue() * 0.01  # Risk 1%
                size = int(risk_amount / risk_per_share)

                if size > 0:
                    self.buy(size=size)
                    self.trades_today += 1
            elif self.position.size < 0:
                self.close()
                self.trades_today += 1

        elif signal == -1 and signal_valid and is_trend_aligned:
            if not self.position:
                atr_value = self.atr[0]
                price = self.data.close[0]
                risk_per_share = atr_value * 1.5
                risk_amount = self.broker.getvalue() * 0.01
                size = int(risk_amount / risk_per_share)

                if size > 0:
                    self.sell(size=size)
                    self.trades_today += 1
            elif self.position.size > 0:
                self.close()
                self.trades_today += 1

        # CRITICAL: DON'T CLOSE ON NEUTRAL SIGNAL
        # Only use trailing stop or time-based exit
        if self.position and self.p.use_trailing_stop:
            if self.position.size > 0:  # Long
                stop_price = self.position.price * (1 - self.p.trail_percent)
                if self.data.close[0] < stop_price:
                    self.close()
            elif self.position.size < 0:  # Short
                stop_price = self.position.price * (1 + self.p.trail_percent)
                if self.data.close[0] > stop_price:
                    self.close()



  cerebro = bt.Cerebro()
  cerebro.addstrategy(MyStrategy)
  # Feed your own dataframe
  datafeed = PandasDataCustom(dataname=data)   # your pandas DataFrame
  cerebro.adddata(datafeed)
  start_cash = 100000
  cerebro.broker.set_cash(start_cash)             # starting capital
  cerebro.broker.setcommission(commission=0.001)
  cerebro.addanalyzer(bt.analyzers.SharpeRatio,
                    _name="sharpe",
                    timeframe=bt.TimeFrame.Days,
                    annualize=True,
                    riskfreerate=0.0)
  # ADD THIS analyzer before cerebro.run():
  cerebro.addanalyzer(bt.analyzers.TimeReturn, _name="timereturn")
  cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="trades")
  cerebro.addanalyzer(bt.analyzers.DrawDown, _name="drawdown")
  cerebro.addanalyzer(bt.analyzers.Returns, _name="returns")
  results = cerebro.run()
  strat = results[0]
  cerebro.plot()
  sharpe = strat.analyzers.sharpe.get_analysis()
  trades = strat.analyzers.trades.get_analysis()
  drawdown = strat.analyzers.drawdown.get_analysis()
  returns = strat.analyzers.returns.get_analysis()
  final_value = cerebro.broker.getvalue()
  total_return = (final_value - start_cash) / start_cash * 100
  cagr = returns.get("rnorm100")  # CAGR in %
  max_dd = drawdown.max.drawdown  # % drawdown

  # Check if trades.total exists before accessing closed trades
  total_trades = trades.total.closed if "total" in trades and "closed" in trades.total else 0
  win_rate = (trades.won.total / trades.total.closed * 100 if "total" in trades and "closed" in trades.total and trades.total.closed > 0 else 0)
  avg_pnl = (trades.pnl.net.average / start_cash * 100 if "pnl" in trades and "net" in trades.pnl and "average" in trades.pnl.net and trades.total.closed > 0 else 0)

  summary = {
        "Final Portfolio Value": final_value,
        "Total Return %": total_return,
        "CAGR %": cagr if cagr else None,
        "Sharpe Ratio": sharpe.get("sharperatio", 0),
        "Max Drawdown %": max_dd,
        "Total Trades": total_trades,
        "Win Rate %": win_rate,
        "Avg Trade PnL %": avg_pnl,
    }
  if total_trades == 0:
      print("No trades were closed during the backtest.")
  return summary
backtest_univariate_trading(df)

{'Final Portfolio Value': 127502.73568508486,
 'Total Return %': 27.50273568508486,
 'CAGR %': 2.238269634636267,
 'Sharpe Ratio': 0.3857248769967194,
 'Max Drawdown %': 16.950706738287106,
 'Total Trades': 86,
 'Win Rate %': 19.767441860465116,
 'Avg Trade PnL %': 0.3182479866257975}

In [ ]:
import backtrader as bt
def backtest_univariate_trading(data):
  class PandasDataCustom(bt.feeds.PandasData):
    lines = ('status',)
    params = (
    ('status', 'status'),
    ('open', 'Open'),
    ('high', 'High'),
    ('low', 'Low'),
    ('close', 'Close'),
    ('volume', 'Volume'),
    ('datetime', None),
    ('openinterest', -1)
    )




  class MyStrategy(bt.Strategy):
    def next(self):
        signal = self.datas[0].status[0]
        if signal == 1 :
          if not self.position:
            self.buy()
          elif self.position.size < 0:
                self.close()
          else:
            pass

        elif signal == -1 :
          if not self.position:
            self.sell()
          elif self.position.size > 0:
             self.close()
          else:
            pass

        else :
          if self.position:
                self.close()
          else:
            pass

  cerebro = bt.Cerebro()
  cerebro.addstrategy(MyStrategy)
  # Feed your own dataframe
  datafeed = PandasDataCustom(dataname=data)   # your pandas DataFrame
  cerebro.adddata(datafeed)
  start_cash = 100000
  cerebro.broker.set_cash(start_cash)             # starting capital
  cerebro.broker.setcommission(commission=0.001)
  cerebro.addanalyzer(bt.analyzers.SharpeRatio,
                    _name="sharpe",
                    timeframe=bt.TimeFrame.Days,
                    annualize=True,
                    riskfreerate=0.0)
  # ADD THIS analyzer before cerebro.run():
  cerebro.addanalyzer(bt.analyzers.TimeReturn, _name="timereturn")
  cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="trades")
  cerebro.addanalyzer(bt.analyzers.DrawDown, _name="drawdown")
  cerebro.addanalyzer(bt.analyzers.Returns, _name="returns")
  results = cerebro.run()
  strat = results[0]
  cerebro.plot()
  sharpe = strat.analyzers.sharpe.get_analysis()
  trades = strat.analyzers.trades.get_analysis()
  drawdown = strat.analyzers.drawdown.get_analysis()
  returns = strat.analyzers.returns.get_analysis()
  final_value = cerebro.broker.getvalue()
  total_return = (final_value - start_cash) / start_cash * 100
  cagr = returns.get("rnorm100")  # CAGR in %
  max_dd = drawdown.max.drawdown  # % drawdown

  # Check if trades.total exists before accessing closed trades
  total_trades = trades.total.closed if "total" in trades and "closed" in trades.total else 0
  win_rate = (trades.won.total / trades.total.closed * 100 if "total" in trades and "closed" in trades.total and trades.total.closed > 0 else 0)
  avg_pnl = (trades.pnl.net.average / start_cash * 100 if "pnl" in trades and "net" in trades.pnl and "average" in trades.pnl.net and trades.total.closed > 0 else 0)

  summary = {
        "Final Portfolio Value": final_value,
        "Total Return %": total_return,
        "CAGR %": cagr if cagr else None,
        "Sharpe Ratio": sharpe.get("sharperatio", 0),
        "Max Drawdown %": max_dd,
        "Total Trades": total_trades,
        "Win Rate %": win_rate,
        "Avg Trade PnL %": avg_pnl,
    }
  if total_trades == 0:
      print("No trades were closed during the backtest.")
  return summary
backtest_univariate_trading(df)

{'Final Portfolio Value': 100068.20533602705,
 'Total Return %': 0.06820533602705109,
 'CAGR %': 0.006212009952234166,
 'Sharpe Ratio': 0.21006101880512465,
 'Max Drawdown %': 0.08519210525757376,
 'Total Trades': 821,
 'Win Rate %': 53.10596833130329,
 'Avg Trade PnL %': 8.307592695144118e-05}

In [ ]:
import backtrader as bt
def backtest_univariate_trading(data):
  class PandasDataCustom(bt.feeds.PandasData):
    lines = ('status',)
    params = (
    ('status', 'status'),
    ('open', 'Open'),
    ('high', 'High'),
    ('low', 'Low'),
    ('close', 'Close'),
    ('volume', 'Volume'),
    ('datetime', None),
    ('openinterest', -1)
    )


  class MyStrategy(bt.Strategy):
    """
    Improved strategy with proper position sizing and risk management
    """
    params = (
        ('risk_per_trade', 0.01),    # Risk 1% per trade
        ('stop_loss_pct', 0.01),     # 1% stop loss
        ('take_profit_pct', 0.03),   # 3% take profit (3:1 ratio)
        ('use_bracket_orders', True), # Use OCO bracket orders
    )

    def __init__(self):
        self.orders = []  # Track active orders
        self.entry_price = 0

    def calculate_position_size(self, stop_price):
        """Calculate position size based on risk"""
        account = self.broker.getvalue()
        risk_amount = account * self.p.risk_per_trade

        if self.position:
            return 0  # Already in position

        if self.p.stop_loss_pct > 0:
            stop_distance = abs(self.data.close[0] - stop_price)
            if stop_distance > 0:
                size = int(risk_amount / stop_distance)
                # Ensure minimum size
                min_size = int(account * 0.0001 / self.data.close[0])
                return max(size, min_size)
        return 0

    def notify_order(self, order):
        """Handle order notifications"""
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed]:
            if order.isbuy() or order.issell():
                self.entry_price = order.executed.price

        # Remove from tracking
        if order in self.orders:
            self.orders.remove(order)

    def next(self):
        signal = self.datas[0].status[0]
        current_price = self.data.close[0]

        # CANCEL ANY PENDING ORDERS
        for order in self.orders:
            self.cancel(order)
        self.orders = []

        # ENTRY LOGIC
        if signal != 0 and not self.position:
            if signal == 1:  # BUY SIGNAL
                # Calculate stop and target
                stop_price = current_price * (1 - self.p.stop_loss_pct)
                target_price = current_price * (1 + self.p.take_profit_pct)

                # Calculate position size
                size = self.calculate_position_size(stop_price)

                if size > 0:
                    # Place main entry order
                    main_order = self.buy(size=size)
                    self.orders.append(main_order)

                    # Place bracket orders (OCO)
                    if self.p.use_bracket_orders:
                        stop_order = self.sell(size=size,
                                             exectype=bt.Order.Stop,
                                             price=stop_price,
                                             parent=main_order)
                        target_order = self.sell(size=size,
                                               exectype=bt.Order.Limit,
                                               price=target_price,
                                               parent=main_order)
                        self.orders.extend([stop_order, target_order])

            elif signal == -1:  # SELL SIGNAL
                stop_price = current_price * (1 + self.p.stop_loss_pct)
                target_price = current_price * (1 - self.p.take_profit_pct)

                size = self.calculate_position_size(stop_price)

                if size > 0:
                    main_order = self.sell(size=size)
                    self.orders.append(main_order)

                    if self.p.use_bracket_orders:
                        stop_order = self.buy(size=size,
                                            exectype=bt.Order.Stop,
                                            price=stop_price,
                                            parent=main_order)
                        target_order = self.buy(size=size,
                                              exectype=bt.Order.Limit,
                                              price=target_price,
                                              parent=main_order)
                        self.orders.extend([stop_order, target_order])

        # EXIT LOGIC - Close on opposite signal
        elif self.position:
            if (signal == 1 and self.position.size < 0) or \
               (signal == -1 and self.position.size > 0):
                self.close()



  cerebro = bt.Cerebro()
  cerebro.addstrategy(MyStrategy)
  # Feed your own dataframe
  datafeed = PandasDataCustom(dataname=data)   # your pandas DataFrame
  cerebro.adddata(datafeed)
  start_cash = 100000
  cerebro.broker.set_cash(start_cash)             # starting capital
  cerebro.broker.setcommission(commission=0.001)
  cerebro.addanalyzer(bt.analyzers.SharpeRatio,
                    _name="sharpe",
                    timeframe=bt.TimeFrame.Days,
                    annualize=True,
                    riskfreerate=0.0)
  # ADD THIS analyzer before cerebro.run():
  cerebro.addanalyzer(bt.analyzers.TimeReturn, _name="timereturn")
  cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="trades")
  cerebro.addanalyzer(bt.analyzers.DrawDown, _name="drawdown")
  cerebro.addanalyzer(bt.analyzers.Returns, _name="returns")
  results = cerebro.run()
  strat = results[0]
  cerebro.plot()
  sharpe = strat.analyzers.sharpe.get_analysis()
  trades = strat.analyzers.trades.get_analysis()
  drawdown = strat.analyzers.drawdown.get_analysis()
  returns = strat.analyzers.returns.get_analysis()
  final_value = cerebro.broker.getvalue()
  total_return = (final_value - start_cash) / start_cash * 100
  cagr = returns.get("rnorm100")  # CAGR in %
  max_dd = drawdown.max.drawdown  # % drawdown

  # Check if trades.total exists before accessing closed trades
  total_trades = trades.total.closed if "total" in trades and "closed" in trades.total else 0
  win_rate = (trades.won.total / trades.total.closed * 100 if "total" in trades and "closed" in trades.total and trades.total.closed > 0 else 0)
  avg_pnl = (trades.pnl.net.average / start_cash * 100 if "pnl" in trades and "net" in trades.pnl and "average" in trades.pnl.net and trades.total.closed > 0 else 0)

  summary = {
        "Final Portfolio Value": final_value,
        "Total Return %": total_return,
        "CAGR %": cagr if cagr else None,
        "Sharpe Ratio": sharpe.get("sharperatio", 0),
        "Max Drawdown %": max_dd,
        "Total Trades": total_trades,
        "Win Rate %": win_rate,
        "Avg Trade PnL %": avg_pnl,
    }
  if total_trades == 0:
      print("No trades were closed during the backtest.")
  return summary
backtest_univariate_trading(df)

{'Final Portfolio Value': 36800.69799112552,
 'Total Return %': -63.19930200887448,
 'CAGR %': -8.705049888105837,
 'Sharpe Ratio': -0.357930512049725,
 'Max Drawdown %': 68.51870185986593,
 'Total Trades': 494,
 'Win Rate %': 48.987854251012145,
 'Avg Trade PnL %': -0.12799056192818425}

In [ ]:
!pip install phidata

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 716.9/716.9 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.7/251.7 kB 16.6 MB/s eta 0:00:00
